In [8]:
import dotenv
import os
import tiktoken
import numpy as np

dotenv.load_dotenv(".env")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [6]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)


In [7]:
index_name = "medical-tests-index"
index = pc.Index(index_name)

In [15]:
print(index.describe_index_stats())

{'dimension': 256,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2263}, 'ns1': {'vector_count': 2323}},
 'total_vector_count': 4586}


In [30]:
def embeddingCreator(text,modelName = "o200k_base"):
    encoding = tiktoken.get_encoding(modelName)
    tokens = encoding.encode(text)
    print("Query embedding shape:", len(tokens))
    return tokens


# Function to retrieve top K similar chunks
def retrieve_similar_chunks(query, top_k=5):
    query_embedding = embeddingCreator(query)
    
    # Ensure the embedding is the correct format for Pinecone
    if isinstance(query_embedding, list):
        query_embedding = np.array(query_embedding, dtype=np.float32).flatten().tolist()
        print("Query embedding shape:", len(query_embedding))
    
    response = index.query(
    namespace="ns1",
    vector=query_embedding * 256,
    top_k=2,
    include_values=True,
    include_metadata=True,
    )
        
    print(response)
    
    # Display results
    # for match in results["matches"]:
    #     print(f"Score: {match['score']}")
    #     print(f"Test Name: {match['metadata']['test_name']}")
    #     print(f"Source: {match['metadata']['source']}")
    #     print("----------------------")



In [31]:
# Example usage
query_text = "Urine Protein And Urine Protein Creatinine Ratio"
retrieve_similar_chunks(query_text, top_k=5)

Query embedding shape: 10
Query embedding shape: 10


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 29 Jan 2025 15:27:11 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '480', 'x-pinecone-request-id': '6744863844283869876', 'x-envoy-upstream-service-time': '0', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 2560 does not match the dimension of the index 256","details":[]}
